In [ ]:
from dataclasses import dataclass
import queue
import random
import threading
import time


@dataclass
class QueryTask:
    id: str
    palyload: dict


def query(task: QueryTask):
    print(f"正在处理任务：{task.id}")
    time.sleep(random.uniform(1.0, 2.0))
    return {"task_id": task.id, "status": "ok", "result": task.palyload}


def query_worker(name, q: queue.Queue):
    while True:
        task = q.get()
        if task is None:
            task.done()
            break
        print(f"正在执行任务：{name}")
        # 调用查询任务
        query(task)
        q.task_done()
        print(f"任务完成：{name}")


def main():
    # 任务列表
    TOTAL_TASKS = 100
    task_list = [
        QueryTask(id=str(random.randint(1, 10000)), palyload={f"task{i}": i})
        for i in range(TOTAL_TASKS)
    ]

    q = queue.Queue(5)
    workers = []
    # 创建线程池
    for i in range(5):
        worker_name = f"worker-{i}"
        t = threading.Thread(
            target=query_worker,
            args=(
                worker_name,
                q,
            ),
            daemon=True,
        )
        t.start()
        workers.append(t)

    # 将任务加入队列
    for task in task_list:
        q.put(task)

    # 给每个worker一个停止信号
    for _ in workers:
        q.put(None)

    # 等待队列中所有任务处理完成
    q.join()
    print("所有任务处理完成")


main()


In [ ]:
# 模拟100个下载任务
import queue
import threading
import random
import time


def download_task(task_id):
    print("开始下载任务：", task_id)
    time.sleep(random.randint(1, 3))
    print("下载任务完成：", task_id)


def worker(q: queue.Queue, name: str):
    while True:
        task = q.get()
        # if task is None:
        #     q.task_done()
        #     break
        # # 执行任务
        # print("开始执行任务：", name)
        # download_task(task["id"])
        # q.task_done()
        try:
            if task is None:
                return
            print("开始执行任务：", name)
            download_task(task["id"])
        except Exception as e:
            print(e)
        finally:
            q.task_done()


def main():
    task_list = [{"id": str(random.randint(1, 100))} for _ in range(100)]
    TASK_NUM = 10
    workers = []
    q = queue.Queue(10)

    # 创建线程池 【先准备消费者】
    for i in range(TASK_NUM):
        task_name = f"worker_{i}"
        t = threading.Thread(target=worker, args=(q, task_name), daemon=True)
        t.start()
        workers.append(t)

    # 添加任务【队列满了任务阻塞】
    for task in task_list:
        q.put(task)

    # 给每个worker添加None
    for _ in workers:
        q.put(None)

    # 等待所有任务完成
    q.join()
    print("所有任务完成")


main()


In [ ]:
stop_event = threading.Event()


def process_task(item):
    print(item)
    time.sleep(random.uniform(1.0, 2.0))
    print("done")


def worker(q: queue.Queue):
    # 当获取任务超时，说明队列为空，
    while not stop_event.is_set():
        try:
            item = q.get(timeout=2.0)
            process_task(item)
        except queue.Empty:
            continue
        print(item)
        q.task_done()


# 结束时：
stop_event.set()


In [3]:
import queue
import threading
import time
import random

MAX_RETRIES = 3


def do_download_task(payload):
    """下载任务【重试机制】"""
    print(f"开始下载任务: {payload}")
    time.sleep(random.uniform(1.0, 2.0))
    if random.random() < 0.3:  # 0.3的概率模拟网络异常
        raise RuntimeError("模拟网络异常")
    return {"id": payload["id"], "status": "success"}


def do_worker(q: queue.Queue, name, stop_event: threading.Event, failure_tasks: list):
    while True:
        if stop_event.is_set() and q.empty():
            print(f"{name} 停止工作")
            break
        try:
            task = q.get(timeout=1.0)
        except queue.Empty:
            # 队列为空则结束本轮循环·
            continue
        try:
            result = do_download_task(task)
            print(f"执行{name} 任务结果: {result}")
        except Exception as e:
            print(f"{name} 错误: {e}")
            # 重试次数+1
            task["retries"] += 1
            if task["retries"] > MAX_RETRIES:
                print(f"任务{task['id']} 重试次数已超过最大限制，任务失败")
                # 将失败的任务添加到失败的列表中
                failure_tasks.append(task)
            else:
                print(f"任务{task['id']} 重新加入队列")
                q.put(task)
        finally:
            q.task_done()


def main():
    # 设置事件终止对象
    stop_event = threading.Event()
    q = queue.Queue(2)

    failure_tasks = []
    # 分配工人干活
    workers = []
    for i in range(2):
        worker_name = f"Worker-{i}"
        thread = threading.Thread(
            target=do_worker,
            args=(q, worker_name, stop_event, failure_tasks),
            daemon=True,
        )
        thread.start()
        workers.append(thread)

    # 派发任务
    for i in range(10):
        q.put(
            {
                "id": i,
                "name": f"Task-{i}",
                "retries": 0,
            }
        )
    # 等待任务完成
    q.join()
    # 任务完成后，停止工作
    stop_event.set()

    # 等待所有工作线程完成
    for worker in workers:
        worker.join()

    print("All done")


main()


开始下载任务: {'id': 0, 'name': 'Task-0', 'retries': 0}开始下载任务: {'id': 1, 'name': 'Task-1', 'retries': 0}

执行Worker-1 任务结果: {'id': 1, 'status': 'success'}
开始下载任务: {'id': 2, 'name': 'Task-2', 'retries': 0}
执行Worker-0 任务结果: {'id': 0, 'status': 'success'}
开始下载任务: {'id': 3, 'name': 'Task-3', 'retries': 0}
执行Worker-1 任务结果: {'id': 2, 'status': 'success'}
开始下载任务: {'id': 4, 'name': 'Task-4', 'retries': 0}
执行Worker-0 任务结果: {'id': 3, 'status': 'success'}
开始下载任务: {'id': 5, 'name': 'Task-5', 'retries': 0}
执行Worker-1 任务结果: {'id': 4, 'status': 'success'}
开始下载任务: {'id': 6, 'name': 'Task-6', 'retries': 0}
执行Worker-0 任务结果: {'id': 5, 'status': 'success'}
开始下载任务: {'id': 7, 'name': 'Task-7', 'retries': 0}
执行Worker-0 任务结果: {'id': 7, 'status': 'success'}
开始下载任务: {'id': 8, 'name': 'Task-8', 'retries': 0}
Worker-1 错误: 模拟网络异常
任务6 重新加入队列
开始下载任务: {'id': 9, 'name': 'Task-9', 'retries': 0}
Worker-0 错误: 模拟网络异常
任务8 重新加入队列
开始下载任务: {'id': 6, 'name': 'Task-6', 'retries': 1}
执行Worker-1 任务结果: {'id': 9, 'status': 'success'}
开始下载

In [ ]:
# 如何处理任务


@dataclass
class TaskResult:
    id: int
    status: str
    data: dict | None = None
    error: str | None = None


def download_task():
    return {"status": "success"}


def worker_task(task_q: queue.Queue, result_q: queue.Queue):
    while True:
        try:
            task = task_q.get()
            if task is None:
                return
            result = download_task()
            task_result = TaskResult(id=task.id, status="success", data=result)
            # 存储成功的任务结果
            result_q.put(task_result)
        except Exception as e:
            # 存储失败的任务结果
            task_result = TaskResult(id=task.id, status="error", error=str(e))
            result_q.put(task_result)
        finally:
            task_q.task_done()


# 如何处理返回的任务队列？
def result_collector(result_q: queue.Queue, stop_event: threading.Event):
    """集中处理：更新JSON/数据库/统计/通知"""
    # 循环队列中的任务:停止条件，发出停止任务信号，同时队列为空
    while not (stop_event.is_set() and result_q.empty()):
        try:
            # 每隔0.5秒取一个任务，如果超时，则继续循环
            task_result = result_q.get(timeout=0.5)
        except queue.Empty:
            continue

        # 任务取出后，处理任务
        try:
            handle_task_result(task_result)
        finally:
            result_q.task_done()


def handle_task_result(task_result: TaskResult): ...
